# SSD512 Test

In [7]:
import tensorflow as tf
import os
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
from scipy.misc import imread
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import copy
from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization
from data_generator.object_detection_2d_data_generator import DataGenerator
from eval_utils.average_precision_evaluator import Evaluator

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

In [2]:
# Set a few configuration parameters.
img_height = 512
img_width = 512
n_classes = 13
model_mode = 'inference'
K.clear_session()

In [3]:
K.clear_session() 

#nastaveni modelu a nacteni vah
model = ssd_512(image_size=(img_height, img_width, 3),
                n_classes=n_classes,
                mode=model_mode,
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
               two_boxes_for_ar1=True,
               steps=[8, 16, 32, 64, 128, 256, 512],
               offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
               clip_boxes=False,
               variances=[0.1, 0.1, 0.2, 0.2],
               normalize_coords=True,
               subtract_mean=[123, 117, 104],
               swap_channels=[2, 1, 0],
               confidence_thresh=0.2,
               iou_threshold=0.45,
               top_k=200,
               nms_max_output_size=400)



weights_path = 'training_summaries/ssd512/ssd512_coralclef2020_epoch-200.h5'

model.load_weights(weights_path, by_name=True)


adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [4]:
test_path = '../CoralClef2020/imageCLEFcoral2020_test_v4/'
img_names = os.listdir(test_path)
print('Pocet testovacich obrazku:', len(img_names))
resize = Resize(height=img_height, width=img_width)

Pocet testovacich obrazku: 400


In [5]:
classes = ['BG','c_hard_coral_branching', 'c_hard_coral_submassive', 'c_hard_coral_boulder',
           'c_hard_coral_encrusting', 'c_hard_coral_table', 'c_hard_coral_foliose', 'c_hard_coral_mushroom',
           'c_soft_coral', 'c_soft_coral_gorgonian', 'c_sponge', 'c_sponge_barrel', 'c_fire_coral_millepora',
           'c_algae_macro_or_leaves']

In [ ]:
data = []

progbar = tf.keras.utils.Progbar(len(img_names))
k = 0

for img_name in img_names:
    progbar.update(k)
    # nacteni obrazku
    filename = img_name
    
    img = np.array(Image.open(test_path + filename), dtype=np.uint8) 
    img2  = resize(img)
    images = img2.reshape([1,512,512,3])

    # detekce
    y_pred = model.predict(images)
    confidence_threshold = 0.5
    y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
    
    # pro odevzdani je potreba pro kazdy obrazek detekovat alespon jeden bbox
    # pokud nebyl detekovan zadny bbox s hodnotou confidence_threshold >= 0.5
    # snizujeme confidence_threshold dokud neziskame bbox
    while(y_pred_thresh[0].shape[0] == 0):
        confidence_threshold = confidence_threshold - 0.1
        y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
        if confidence_threshold <= 0.2:
            break
    print(confidence_threshold)
    
    # zmena velikosti ramecku
    y_pred_resized = copy.deepcopy(y_pred_thresh)
    for r in y_pred_resized[0]:
        r[2] = r[2] * (img.shape[1]/img_width)
        r[3] = r[3] * (img.shape[0]/img_height)
        r[4] = r[4] * (img.shape[1]/img_width)
        r[5] = r[5] * (img.shape[0]/img_height)
        
    # konverze dat do pozadovaneho formatu
    row = convert_data(filename, y_pred_resized)
    data.append(row)
    
    k = k + 1

 39/400 [=>............................] - ETA: 1:58

In [54]:
with open('test_SSD.txt','w') as f:
    f.write('\n'.join(data))

In [10]:
def convert_data(filename, y_pred_resized):
    bbox_data = {}
    for box in y_pred_resized[0]:
        class_id = box[0]
        class_name = classes[int(class_id)]
        confidence = round(box[1],1)
        xmin = int(box[2])
        ymin = int(box[3])
        xmax = int(box[4])
        ymax = int(box[5])
        if(xmin < 0):
            xmin = 0
        if(ymin < 0):
            ymin = 0
        if(ymax > 3024):
            ymax = 3024
        if(xmax > 4032):
            xmax = 4032
        width = xmax - xmin
        height = ymax - ymin


        if(class_name not in bbox_data.keys()):
            bbox_data.update({class_name:[[confidence,width,height,xmin,ymin]]})
        else:
            bbox_data[class_name].append([confidence,width,height,xmin,ymin])

    row = filename.split('.')[0] + ';'
    k = 1
    for c in bbox_data.keys():
        row = row + c + ' '
        i = 1
        for b in bbox_data[c]:
            if(i < len(bbox_data[c])):
                row = row + '{:.1f}:{}x{}+{}+{},'.format(b[0], b[1], b[2], b[3], b[4])
                i = i + 1
            else:
                row = row + '{:.1f}:{}x{}+{}+{}'.format(b[0], b[1], b[2], b[3], b[4])
        if(k < len(bbox_data.keys())):
            row = row + ';'
            k = k + 1
    return row